In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .master("spark://iceberg-sandbox-spark-master:7077")
    .appName("iceberg-sandbox")
    .config(
        "spark.jars",
        "/opt/extra-jars/iceberg-spark-runtime-3.5_2.12-1.5.2.jar,"
        "/opt/extra-jars/hadoop-aws-3.3.4.jar,"
        "/opt/extra-jars/aws-java-sdk-bundle-1.12.262.jar,"
        "/opt/extra-jars/postgresql-42.7.3.jar"
    )
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.iceberg_jdbc", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.iceberg_jdbc.catalog-impl", "org.apache.iceberg.jdbc.JdbcCatalog")
    .config("spark.sql.catalog.iceberg_jdbc.uri", "jdbc:postgresql://iceberg-sandbox-postgres:5432/iceberg-jdbc-catalog")
    .config("spark.sql.catalog.iceberg_jdbc.jdbc.user", "iceberg")
    .config("spark.sql.catalog.iceberg_jdbc.jdbc.password", "iceberg")
    .config("spark.sql.catalog.iceberg_jdbc.jdbc.driver", "org.postgresql.Driver")
    .config("spark.sql.catalog.iceberg_jdbc.warehouse", "s3a://iceberg/warehouse")
    .config("spark.hadoop.fs.s3a.endpoint", "http://iceberg-sandbox-minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)


In [1]:
from pyspark.sql import SparkSession 
spark = ( SparkSession.builder 
         .master("spark://iceberg-sandbox-spark-master:7077") 
         .appName("iceberg-sandbox") # Distribute jars to cluster 
         .config( "spark.jars", 
                 "/opt/extra-jars/iceberg-spark-runtime-3.5_2.12-1.5.2.jar," 
                 "/opt/extra-jars/hadoop-aws-3.3.4.jar," 
                 "/opt/extra-jars/aws-java-sdk-bundle-1.12.262.jar," 
                 "/opt/extra-jars/postgresql-42.7.3.jar" ) # Required Iceberg extension 
         .config( "spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions" ) # 🔥 JDBC Catalog instead of Hadoop 
         .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") 
         .config( "spark.sql.catalog.iceberg.catalog-impl", "org.apache.iceberg.jdbc.JdbcCatalog" ) 
         .config( "spark.sql.catalog.iceberg.uri", "jdbc:postgresql://iceberg-sandbox-postgres:5432/iceberg-jdbc-catalog" ) 
         .config("spark.sql.catalog.iceberg.jdbc.user", "iceberg") 
         .config("spark.sql.catalog.iceberg.jdbc.password", "iceberg") 
         .config("spark.sql.catalog.iceberg.jdbc.driver", "org.postgresql.Driver") 
         .config("spark.sql.catalog.iceberg.catalog-name", "iceberg_jdbc") # Warehouse in MinIO 
         .config( "spark.sql.catalog.iceberg.warehouse", "s3a://iceberg/warehouse" ) # MinIO config 
         .config("spark.hadoop.fs.s3a.endpoint", "http://iceberg-sandbox-minio:9000") 
         .config("spark.hadoop.fs.s3a.access.key", "minioadmin") 
         .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") 
         .config("spark.hadoop.fs.s3a.path.style.access", "true") 
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") 
         .getOrCreate() )

In [21]:
spark.sql("SHOW NAMESPACES IN iceberg_jdbc").show()

+-----------------+
|        namespace|
+-----------------+
|test_schema_trino|
+-----------------+



In [ ]:
spark

In [16]:
spark.sql("SHOW CATALOGS").show()

+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [22]:
spark.sql("SHOW SCHEMAS FROM iceberg_jdbc").show()

+-----------------+
|        namespace|
+-----------------+
|test_schema_trino|
+-----------------+



In [14]:
spark.sql("SHOW NAMESPACES IN iceberg").show()

AnalysisException: [SCHEMA_NOT_FOUND] The schema `iceberg` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a catalog, verify the current_schema() output, or qualify the name with the correct catalog.
To tolerate the error on drop use DROP SCHEMA IF EXISTS.

In [13]:
spark.sql("SHOW SCHEMAS FROM iceberg").show()

AnalysisException: [SCHEMA_NOT_FOUND] The schema `iceberg` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a catalog, verify the current_schema() output, or qualify the name with the correct catalog.
To tolerate the error on drop use DROP SCHEMA IF EXISTS.

In [10]:
spark.conf.get("spark.sql.catalog.iceberg.catalog-name")

'iceberg_jdbc'

In [ ]:
spark.sql("DESCRIBE NAMESPACE iceberg.test_schema_trino").show()


In [8]:
spark.sql("""
CREATE TABLE iceberg_jdbc.test_schema_trino.users_spark (
    id INT,
    name STRING
)
USING iceberg
""")

DataFrame[]

In [9]:
spark.sql("""
INSERT INTO iceberg_jdbc.test_schema_trino.users_spark
VALUES (1, 'Fajar'), (2, 'Spark')
""")


DataFrame[]

In [18]:
spark.stop()